<a href="https://colab.research.google.com/github/ary3120-droid/myproject/blob/main/fraud1119.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 로드& 데이터 탐색

In [128]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [129]:
df = pd.read_csv('fraud.csv')
print(df.shape)
df.head()

(491134, 22)


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


컬럼명 정리

In [130]:
#현재 컬럼명 리스트 확인
dating_df.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

trans_date_trans_time — 거래 날짜·시간

cc_num — 카드 번호

merchant — 가맹점 이름

category — 가맹점 카테고리

amt — 거래 금액

first — 카드 소유자 이름(이름)

last — 카드 소유자 성

gender — 성별

street — 거주지 도로명 주소

city — 거주 도시

state — 거주 주

zip — 우편번호

lat — 카드 소유자 집 위도

long — 카드 소유자 집 경도

city_pop — 해당 도시 인구

job — 직업

dob — 생년월일

trans_num — 거래 고유 ID

unix_time — 유닉스 시간

merch_lat — 가맹점 위도

merch_long — 가맹점 경도

is_fraud — 사기 여부(0 정상 / 1 사기)

In [131]:
# 현재 컬럼명 리스트 확인
df.columns

# 불필요한 컬럼 제거
drop_cols = [
    'first', 'last',
    'street', 'city', 'state', 'zip',
    'dob', 'job',
    'cc_num', 'trans_num'
]

df = df.drop(columns=drop_cols)


결측치 처리

In [132]:
df.isnull().sum()

,0
trans_date_trans_time,0
merchant,0
category,0
amt,0
gender,0
lat,0
long,0
city_pop,0
unix_time,0
merch_lat,0


In [133]:
df = df.dropna(subset=['trans_date_trans_time'])
df['trans_dt'] = pd.to_datetime(df['trans_date_trans_time'])
df['hour'] = df['trans_dt'].dt.hour
df['weekday'] = df['trans_dt'].dt.weekday
df['is_weekend'] = (df['weekday'] >= 5).astype(int)


In [134]:
# 수치형 컬럼은 중앙값으로
num_cols = df.select_dtypes(include=['float64', 'int64', 'int32']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
 #금액/위치/시간 데이터는 대부분
 #왜곡되어 있어서 평균(mean)보다 중앙값이 안정적

In [135]:
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)


In [136]:
df.isnull().sum()


,0
trans_date_trans_time,0
merchant,0
category,0
amt,0
gender,0
lat,0
long,0
city_pop,0
unix_time,0
merch_lat,0


피처 엔지니어링(Feature Engineering)

**1.시간 기반 피처**



In [137]:
#1) 거래가 일어난 시간대(hour)
#야간 거래(0–6시)에 사기가 많다는 실전 패턴이 자주 나옴.

df['hour'] = df['trans_dt'].dt.hour


In [138]:
#2) 요일(weekday)
#주말/평일 패턴 반영.

df['weekday'] = df['trans_dt'].dt.weekday


In [139]:
#3) 주말 여부
df['is_weekend'] = (df['weekday'] >= 5).astype(int)


In [140]:
#4) 시간대 구간화 #새벽/낮/저녁/밤

def time_period(h):
    if 0 <= h < 6:
        return "dawn"
    elif 6 <= h < 12:
        return "morning"
    elif 12 <= h < 18:
        return "afternoon"
    else:
        return "night"

df['time_period'] = df['hour'].apply(time_period)


In [141]:
df = df.drop(columns=['trans_date_trans_time', 'trans_dt'])


**2. 위치 기반 피처**

In [142]:
#1) 고객 집 ↔ 가맹점 거리(distance)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

df['distance'] = haversine(
    df['lat'], df['long'],
    df['merch_lat'], df['merch_long']
)


**3. 금액 기반 피처**

In [143]:
df['log_amount'] = np.log1p(df['amt'])
#1) 로그 변환

In [144]:
df['high_amount_flag'] = (df['amt'] > 2000).astype(int)
#2) 고액 거래 여부

인코딩

In [145]:
le = LabelEncoder()
df['merchant'] = le.fit_transform(df['merchant'])
#merchant는 Label Encoding

In [146]:
#category, gender, time_period는 원핫 인코딩
df = pd.get_dummies(
    df,
    columns=['category', 'gender', 'time_period'],
    drop_first=True,
    dtype=int
)


In [147]:
#최종 타입 확인
print(df.dtypes)
print("숫자 아닌 컬럼:", df.select_dtypes(exclude=['number']).columns)


merchant                     int64
amt                        float64
lat                        float64
long                       float64
city_pop                     int64
unix_time                    int64
merch_lat                  float64
merch_long                 float64
is_fraud                     int64
hour                         int32
weekday                      int32
is_weekend                   int64
distance                   float64
log_amount                 float64
high_amount_flag             int64
category_food_dining         int64
category_gas_transport       int64
category_grocery_net         int64
category_grocery_pos         int64
category_health_fitness      int64
category_home                int64
category_kids_pets           int64
category_misc_net            int64
category_misc_pos            int64
category_personal_care       int64
category_shopping_net        int64
category_shopping_pos        int64
category_travel              int64
gender_M            

학습/테스트 데이터 분리

In [148]:

X = df.drop(columns=['is_fraud'])
y = df['is_fraud'].astype(int)

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y   # 클래스 불균형이 있어 stratify
)


모델 학습 (RandomForest + class_weight로 불균형 고려)

In [149]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    class_weight='balanced',
    random_state=42
)

model.fit(X_train, y_train)


KeyboardInterrupt: 

모델평가

In [ ]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

feat_imp = pd.Series(model.feature_importances_, index=X_train.columns)
feat_imp.nlargest(20).sort_values().plot(kind='barh', figsize=(6,8))
plt.title("Feature Importance")
plt.show()
